In [359]:
import numpy as np
import pandas as pd
import math


In [3]:
arr_data = np.array([['-', 'short', 'blond', 'brown'],
                 ['-', 'tall', 'dark', 'brown'],
                 ['+', 'tall', 'blond', 'blue'],
                 ['-', 'tall', 'dark', 'blue'],
                 ['-', 'short', 'dark', 'blue'],
                 ['+', 'tall', 'red', 'blue'],
                 ['-', 'tall', 'blond', 'brown'],
                 ['+', 'short', 'blond', 'blue']])


data = pd.DataFrame(arr_data, columns = ['class', 'height', 'hair', 'eyes'])
data

,class,height,hair,eyes
0,-,short,blond,brown
1,-,tall,dark,brown
2,+,tall,blond,blue
3,-,tall,dark,blue
4,-,short,dark,blue
5,+,tall,red,blue
6,-,tall,blond,brown
7,+,short,blond,blue


In [487]:
class TreeNode:
    def __init__(self, instance):
        self.instances = instance
        self.isleaf = True
        self.class_labels = instance.iloc[:, 0].mode().iloc[0]
        self.children = {}



class IncrementalDTree:
    def __init__(self):
        self.root = None


    def predict(self, instance, current_node):
        if current_node.isleaf == True:
            return current_node

        else:
            split_attribute_val = instance[current_node.instances]
            try:
                return self.predict(instance.drop(split_attribute_val, axis = 1), current_node.children[split_attribute_val])
            except:
                return 'bad'

    def entropy(self, columns, data, target_attributes):
        frequency = {}
        entropy_data = 0.0
        i = 0
        for entry in columns:
            if (target_attributes == entry):
                break
            i = i + 1
        i = i - 1
        for entry in data:
            if (entry[i] in frequency.keys()):
                frequency[entry[i]] += 1.0
            else:
                frequency[entry[i]]  = 1.0
        for frequency in frequency.values():
            entropy_data += (-frequency/len(data)) * math.log(frequency/len(data), 2)
        return entropy_data

    def informationGain(self, columns, data, attr, target_attributes):
        frequency_of_desired_var = {}
        subset_entropy = 0.0
        i = columns.index(attr)

        for entry in data:
            if (entry[i] in frequency_of_desired_var.keys()):
                frequency_of_desired_var[entry[i]] += 1.0
            else:
                frequency_of_desired_var[entry[i]]  = 1.0

        for val in frequency_of_desired_var.keys():
            valProb        = frequency_of_desired_var[val] / sum(frequency_of_desired_var.values())
            dataSubset     = [entry for entry in data if entry[i] == val]
            subset_entropy += valProb * self.entropy(columns, dataSubset, target_attributes)

        return (self.entropy(columns, data, target_attributes) - subset_entropy)

                    

    def find_split(self, node):
        columns = list(node.columns)
        best = columns[0]
        # And maximum information gain to be 0
        maximim_gain = 0;

        for attr in columns:
            # For each columnn find out the information gain
            new_info_gain = self.informationGain(columns, data, attr, columns[0])
            # If the new information gain happens to be more than the current maximum
            # Then update the current max info_gain with the new gain
            if new_info_gain>maximim_gain:
                maximim_gain = new_info_gain
                best = attr


        return best

    def add_node(self, instance, current_node):
        if current_node.isleaf == True:
            pass

        else:
            split_attribute_val = instance[current_node.instances].iloc[0] 
            try:
                self.add_node(instance.drop(split_attribute_val, axis = 1), current_node.children[split_attribute_val])
            except:
                new_node = TreeNode(instance)
                print(split_attribute_val)
                current_node.children[split_attribute_val] = new_node



    def split(self, node):
        split_feat = self.find_split(node.instances)
        print(split_feat)
        # split_feat = node.instances.columns[split_feat]
        if split_feat == None:
            return node

        else:
            print(split_feat, '\n', node.instances)
            for val in node.instances[split_feat]:
                temp_instances = node.instances.drop(split_feat, axis = 1)
                new_node = TreeNode(temp_instances[node.instances[split_feat] == val])
                node.children[val] = new_node
            node.instances = split_feat
            node.isleaf = False
            print(node.instances, node.children)
            return node

    def fit_instance(self, instance, label):
        if self.root == None:
            new_node = TreeNode(instance)
            self.root = new_node

        elif self.root != None:
            pred_node = self.predict(instance, self.root)
            if pred_node != 'bad':
                pred = pred_node.class_labels
                print(pred, label)
                if pred == label:
                    # Could add the the instances to the node but skipping for now
                    pass
                else:
                    pred_node.instances = pd.concat([pred_node.instances, instance[pred_node.instances.columns]])
                    pred_node = self.split(pred_node)
                    print(pred_node.instances, pred_node.children)
                    print(self.root.instances, self.root.children)
            else:
                print(self.root.children)
                self.add_node(instance, self.root)
                print(self.root.children)

    def print_tree(self, current_node):
        if current_node.isleaf == True:
            print(current_node.class_labels, end = '\t')
        elif current_node.isleaf != True:
            print(current_node.instances, end = '\t')
            for node in current_node.children:
                self.print_tree(current_node.children[node])
                print()
        # except Exception as e:
        #     print(e)
        #     print(current_node)
                    


                    

            

In [488]:
tree = IncrementalDTree()

In [489]:
tree.fit_instance(data.iloc[[0]], data.iloc[0, 0])

In [490]:
tree.print_tree(tree.root)

-	

In [491]:
tree.fit_instance(data.iloc[[1]], data.iloc[1, 0])

- -


In [492]:
tree.predict(data.iloc[[1]], tree.root).class_labels

'-'

In [493]:
tree.print_tree(tree.root)

-	

In [494]:
tree.fit_instance(data.iloc[[2]], data.iloc[2, 0])

- +
height
height 
   class height   hair   eyes
0     -  short  blond  brown
2     +   tall  blond   blue
height {'short': <__main__.TreeNode object at 0x1177c1640>, 'tall': <__main__.TreeNode object at 0x1177c1ee0>}
height {'short': <__main__.TreeNode object at 0x1177c1640>, 'tall': <__main__.TreeNode object at 0x1177c1ee0>}
height {'short': <__main__.TreeNode object at 0x1177c1640>, 'tall': <__main__.TreeNode object at 0x1177c1ee0>}


In [495]:
tree.print_tree(tree.root)

height	-	
+	


In [496]:
tree.fit_instance(data.iloc[[3]], data.iloc[3, 0])

{'short': <__main__.TreeNode object at 0x1177c1640>, 'tall': <__main__.TreeNode object at 0x1177c1ee0>}
tall
{'short': <__main__.TreeNode object at 0x1177c1640>, 'tall': <__main__.TreeNode object at 0x1177c1280>}


In [484]:
tree.print_tree(tree.root)

height	-	
-	


In [485]:
tree.root.instances

'height'

In [486]:
tree.root.children

{'short': <__main__.TreeNode at 0x117665880>,
 'tall': <__main__.TreeNode at 0x11775efd0>}

In [461]:
tree.root.children['short'].instances

,class,hair,eyes
0,-,blond,brown


In [462]:
tree.root.children['tall'].instances

,class,height,hair,eyes
3,-,tall,dark,blue


In [463]:
tree.root.class_labels

'-'

In [464]:
pd.concat([data.iloc[[0]], data.iloc[[-1]]])

,class,height,hair,eyes
0,-,short,blond,brown
7,+,short,blond,blue


In [465]:
data.iloc[0].iloc[0]

'-'

In [466]:
temp_instances = data.drop('height', axis = 1)
temp_instances[data['height'] == 'tall']

,class,hair,eyes
1,-,dark,brown
2,+,blond,blue
3,-,dark,blue
5,+,red,blue
6,-,blond,brown


In [467]:
tree = IncrementalDTree()

for i in range(data.shape[0]):
    tree.fit_instance(data.iloc[[i]], data.iloc[i, 0])

- -
- +
height
height 
   class height   hair   eyes
0     -  short  blond  brown
2     +   tall  blond   blue
height {'short': <__main__.TreeNode object at 0x1173ed0d0>, 'tall': <__main__.TreeNode object at 0x1173ed280>}
height {'short': <__main__.TreeNode object at 0x1173ed0d0>, 'tall': <__main__.TreeNode object at 0x1173ed280>}
height {'short': <__main__.TreeNode object at 0x1173ed0d0>, 'tall': <__main__.TreeNode object at 0x1173ed280>}
tall
short
tall
tall
short


In [468]:
tree.print_tree(tree.root)

height	+	
-	


In [469]:
tree.root.instances

'height'

In [470]:
tree.root.children

{'short': <__main__.TreeNode at 0x1173edd60>,
 'tall': <__main__.TreeNode at 0x1174e4130>}

In [471]:
tree.root.children['short'].children

{}

In [472]:
tree.root.children['tall'].children

{}

In [473]:
for i in range(data.shape[0]):
    print(data.iloc[i, 0], tree.predict(data.iloc[[i]], tree.root))

- bad
- bad
+ bad
- bad
- bad
+ bad
- bad
+ bad


In [447]:
data['class'].mode().iloc[0]

'-'